In [1]:
import threading #the python threading library
import time #the python time library
from pynq.overlays.base import BaseOverlay #FPGA bitstream (hardware file)
base = BaseOverlay("base.bit") # " " " "
import pynq.lib.rgbled as rgbled #for philosopher #4

In [2]:
#configure and test buttons:
btns = base.btns_gpio

#test code:
'''
while True:
    if btns.read() == 1:
        base.leds[0].on()
        base.leds[1].off()
        base.leds[2].off()
        base.leds[3].off()
    elif btns.read() == 2:
        base.leds[0].off()
        base.leds[1].on()
        base.leds[2].off()
        base.leds[3].off()
    elif btns.read() == 4:
        base.leds[0].off()
        base.leds[1].off()
        base.leds[2].on()
        base.leds[3].off()
    elif btns.read() == 8:
        base.leds[3].on()
        base.leds[1].off()
        base.leds[2].off()
        base.leds[3].off()
    else:
        base.leds[0].off()
        base.leds[1].off()
        base.leds[2].off()
        base.leds[3].off()
'''

'\nwhile True:\n    if btns.read() == 1:\n        base.leds[0].on()\n        base.leds[1].off()\n        base.leds[2].off()\n        base.leds[3].off()\n    elif btns.read() == 2:\n        base.leds[0].off()\n        base.leds[1].on()\n        base.leds[2].off()\n        base.leds[3].off()\n    elif btns.read() == 4:\n        base.leds[0].off()\n        base.leds[1].off()\n        base.leds[2].on()\n        base.leds[3].off()\n    elif btns.read() == 8:\n        base.leds[3].on()\n        base.leds[1].off()\n        base.leds[2].off()\n        base.leds[3].off()\n    else:\n        base.leds[0].off()\n        base.leds[1].off()\n        base.leds[2].off()\n        base.leds[3].off()\n'

In [6]:
#Declare philosopher variables:
phil0 = 0
phil1 = 1
phil2 = 2
phil3 = 3
phil4 = 4
rgb_code_green = 0x2
rgb_code_off = 0x0

#Declare timing variables:
time_EATING = 1
duration_EATING = 0.02
time_NAPPING = 3
duration_NAPPING = 0.2
time_STARVING = 1

btns = base.btns_gpio

def blink(times_to_toggle, on_off_duration, LED_index): 
    '''
    Function to blink the LEDs
    Params:
      times_to_toggle: number of times to toggle the LED from on and off
      on_off_duration: duration (in seconds) for the LED to be on/off
      LED_index: index of the LED (0 to 3)
      
      total blink time (sec) = times_to_toggle*on_off_duration
    '''
    if LED_index < 4:
        for i in range(times_to_toggle):
            base.leds[LED_index].toggle()
            time.sleep(on_off_duration)
        base.leds[LED_index].off()
    else: #dealing with RGB LED then
        for i in range(times_to_toggle):
            if i % 2 == 0:
                rgbled.RGBLED(4).write(rgb_code_green)
            else:
                rgbled.RGBLED(4).write(rgb_code_off)
            time.sleep(on_off_duration)
        rgbled.RGBLED(4).write(0x0)
    
def eating(philosopher_index):
    blink(int(time_EATING/duration_EATING), duration_EATING, philosopher_index)
    
def napping(philosopher_index):
    blink(int(time_NAPPING/duration_NAPPING), duration_NAPPING, philosopher_index)
    
#test I have the right LEDs connected by running the following sequence:
'''
blink(25, 0.02, phil0)
blink(25, 0.02, phil1)
blink(25, 0.02, phil2)
blink(25, 0.02, phil3)
blink(25, 0.02, phil4)
#print(time_EATING/duration_EATING) #turns out this is a float prob because duration_EATING is a float
eating(phil0)
napping(phil0)
eating(phil1)
napping(phil1)
eating(phil2)
napping(phil2)
eating(phil3)
napping(phil3)
eating(phil4)
napping(phil4)
'''

def get_btns_state():
    return btns.read()

def worker_phil0(wp0_lock40, wp0_lock01):
    fork40_available = False
    fork01_available = False
    while get_btns_state() == 0: #if no the buttons are pushed...
        fork40_available = wp0_lock40.acquire(False) #non-blocking acquire()
        fork01_available = wp0_lock01.acquire(False) #" " "
        if fork40_available and fork01_available:
            print("Philosopher 0 has started eating") # print message for having both forks in hand
            eating(phil0)
            wp0_lock40.release() # release the forks
            wp0_lock01.release() # " " " 
            napping(phil0) #philosopher naps after the meal
            #time.sleep(0.5) # give enough time to the other thread to grab the key
        else:
            print("Philosopher 0 is starving...") # print message indicating both forks are not available
            #time.sleep(time_STARVING) #philosopher patiently waits time_STARVING before checking both forks again...
    print('Program terminated. Press Shift + Enter in the cell to start philosophers` feast again.')

def worker_phil1(lock01, lock12):
    fork01_available = False
    fork12_available = False
    while get_btns_state() == 0: #if no the buttons are pushed...
        fork01_available = lock01.acquire(False) #non-blocking acquire()
        fork12_available = lock12.acquire(False) #" " "
        if fork01_available and fork12_available:
            print("Philosopher 1 has started eating") # print message for having both forks in hand
            eating(phil1)
            lock01.release() # release the forks
            lock12.release() # " " " 
            napping(phil1) #philosopher naps after the meal
            #time.sleep(0.5) # give enough time to the other thread to grab the key
        else:
            print("Philosopher 1 is starving...") # print message indicating both forks are not available
            #time.sleep(time_STARVING) #philosopher patiently waits time_STARVING before checking both forks again...  
    print('Program terminated. Press Shift + Enter in the cell to start philosophers` feast again.')

def worker_phil2(lock12, lock23):
    fork12_available = False
    fork23_available = False
    while get_btns_state() == 0: #if no the buttons are pushed...
        fork12_available = lock12.acquire(False) #non-blocking acquire()
        fork23_available = lock23.acquire(False) #" " "
        if fork12_available and fork23_available:
            print("Philosopher 2 has started eating") # print message for having both forks in hand
            eating(phil2)
            lock12.release() # release the forks
            lock23.release() # " " " 
            napping(phil2) #philosopher naps after the meal
            #time.sleep(0.5) # give enough time to the other thread to grab the key
        else:
            print("Philosopher 2 is starving...") # print message indicating both forks are not available
            #time.sleep(time_STARVING) #philosopher patiently waits time_STARVING before checking both forks again... 
    print('Program terminated. Press Shift + Enter in the cell to start philosophers` feast again.')

def worker_phil3(lock23, lock34):
    fork23_available = False
    fork34_available = False
    while get_btns_state() == 0: #if no the buttons are pushed...
        fork23_available = lock23.acquire(False) #non-blocking acquire()
        fork34_available = lock34.acquire(False) #" " "
        if fork23_available and fork34_available:
            print("Philosopher 3 has started eating") # print message for having both forks in hand
            eating(phil3)
            lock23.release() # release the forks
            lock34.release() # " " " 
            napping(phil3) #philosopher naps after the meal
            #time.sleep(0.5) # give enough time to the other thread to grab the key
        else:
            print("Philosopher 3 is starving...") # print message indicating both forks are not available
            #time.sleep(time_STARVING) #philosopher patiently waits time_STARVING before checking both forks again... 
    print('Program terminated. Press Shift + Enter in the cell to start philosophers` feast again.')
        
def worker_phil4(lock34, lock40):
    fork34_available = False
    fork40_available = False
    while get_btns_state() == 0: #if no the buttons are pushed...
        fork34_available = lock34.acquire(False) #non-blocking acquire()
        fork40_available = lock40.acquire(False) #" " "
        if fork34_available and fork40_available:
            print("Philosopher 4 has started eating") # print message for having both forks in hand
            eating(phil4)
            lock34.release() # release the forks
            lock40.release() # " " " 
            napping(phil4) #philosopher naps after the meal
            #time.sleep(0.5) # give enough time to the other thread to grab the key
        else:
            print("Philosopher 4 is starving...") # print message indicating both forks are not available
            #time.sleep(time_STARVING) #philosopher patiently waits time_STARVING before checking both forks again...   
    print('Program terminated. Press Shift + Enter in the cell to start philosophers` feast again.')
        
threads = []
fork01 = threading.Lock() #the fork between philosophers 0 and 1
fork12 = threading.Lock() #the fork between philosophers 1 and 2
fork23 = threading.Lock() #the fork between philosophers 2 and 3
fork34 = threading.Lock() #the fork between philosophers 3 and 4
fork40 = threading.Lock() #the fork between philosophers 4 and 0

t = threading.Thread(target=worker_phil0, args=(fork40, fork01))
threads.append(t)
t.start()
t = threading.Thread(target=worker_phil1, args=(fork01, fork12))
threads.append(t)
t.start()
t = threading.Thread(target=worker_phil2, args=(fork12, fork23))
threads.append(t)
t.start()
t = threading.Thread(target=worker_phil3, args=(fork23, fork34))
threads.append(t)
t.start()
t = threading.Thread(target=worker_phil4, args=(fork34, fork40))
threads.append(t)
t.start()

for t in threads:
    t.join()


Philosopher 0 has started eatingPhilosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...Philosopher 2 is starving...

Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...

Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher

Philosopher 4 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...


Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 

Philosopher 3 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 

Philosopher 4 is starving...Philosopher 3 is starving...
Philosopher 3 is starving...

Philosopher 3 is starving...
Philosopher 3 is starving...Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...
Philosopher 2 is starving...

Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 

Philosopher 3 is starving...Philosopher 1 is starving...
Philosopher 1 is starving...

Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 i

...

Philosopher 0 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...

Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 

Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 

Philosopher 2 is starving...

Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...
Philosopher 1 is starving...Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 is starving...
Philosopher 0 i

Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...

Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 4 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3 is starving...
Philosopher 3

Philosopher 4 is starving...Program terminated. Press Shift + Enter in the cell to start philosophers` feast again.

Program terminated. Press Shift + Enter in the cell to start philosophers` feast again.

Program terminated. Press Shift + Enter in the cell to start philosophers` feast again.
Program terminated. Press Shift + Enter in the cell to start philosophers` feast again.

Program terminated. Press Shift + Enter in the cell to start philosophers` feast again.


In [11]:
#Declare philosopher variables:
phil0 = 0
phil1 = 1
phil2 = 2
phil3 = 3
phil4 = 4
rgb_code_green = 0x2
rgb_code_off = 0x0

#Declare timing variables:
time_EATING = 1
duration_EATING = 0.02
time_NAPPING = 2
duration_NAPPING = 0.2
time_STARVING = 1

btns = base.btns_gpio

def blink(times_to_toggle, on_off_duration, LED_index): 
    '''
    Function to blink the LEDs
    Params:
      times_to_toggle: number of times to toggle the LED from on and off
      on_off_duration: duration (in seconds) for the LED to be on/off
      LED_index: index of the LED (0 to 3)
      
      total blink time (sec) = times_to_toggle*on_off_duration
    '''
    if LED_index < 4:
        for i in range(times_to_toggle):
            base.leds[LED_index].toggle()
            time.sleep(on_off_duration)
        base.leds[LED_index].off()
    else: #dealing with RGB LED then
        for i in range(times_to_toggle):
            if i % 2 == 0:
                rgbled.RGBLED(4).write(rgb_code_green)
            else:
                rgbled.RGBLED(4).write(rgb_code_off)
            time.sleep(on_off_duration)
        rgbled.RGBLED(4).write(0x0)
    
def eating(philosopher_index):
    blink(int(time_EATING/duration_EATING), duration_EATING, philosopher_index)
    
def napping(philosopher_index):
    blink(int(time_NAPPING/duration_NAPPING), duration_NAPPING, philosopher_index)
    
#test I have the right LEDs connected by running the following sequence:
'''
blink(25, 0.02, phil0)
blink(25, 0.02, phil1)
blink(25, 0.02, phil2)
blink(25, 0.02, phil3)
blink(25, 0.02, phil4)
#print(time_EATING/duration_EATING) #turns out this is a float prob because duration_EATING is a float
eating(phil0)
napping(phil0)
eating(phil1)
napping(phil1)
eating(phil2)
napping(phil2)
eating(phil3)
napping(phil3)
eating(phil4)
napping(phil4)
'''

def get_btns_state():
    return btns.read()

def worker_phils(lock01, lock12, lock23, lock34, lock40, num):
    p0_fork40_available = False
    p0_fork01_available = False
    p1_fork01_available = False
    p1_fork12_available = False
    p2_fork12_available = False
    p2_fork23_available = False
    p3_fork23_available = False
    p3_fork34_available = False
    p4_fork34_available = False
    p4_fork40_available = False
    
    while get_btns_state() == 0: #if no the buttons are pushed...
        if num == phil0:
            p0_fork40_available = lock40.acquire(False) #non-blocking acquire()
            p0_fork01_available = lock01.acquire(False) #" " "
            if p0_fork40_available and p0_fork01_available:
                print("Philosopher 0 has started eating") # print message for having both forks in hand
                eating(phil0)
                lock40.release() # release the forks
                lock01.release() # " " " 
                napping(phil0) #philosopher naps after the meal
            else:
                print("Philosopher 0 is starving...") # print message indicating both forks are not available
                time.sleep(0.2)
        elif num == phil1:
            p1_fork01_available = lock01.acquire(False) #non-blocking acquire()
            p1_fork12_available = lock12.acquire(False) #" " "
            if p1_fork01_available and p1_fork12_available:
                print("Philosopher 1 has started eating") # print message for having both forks in hand
                eating(phil1)
                lock01.release() # release the forks
                lock12.release() # " " " 
                napping(phil1) #philosopher naps after the meal
            else:
                print("Philosopher 1 is starving...") # print message indicating both forks are not available
                time.sleep(0.2)
        elif num == phil2:
            p2_fork12_available = lock12.acquire(False) #non-blocking acquire()
            p2_fork23_available = lock23.acquire(False) #" " "
            if p2_fork12_available and p2_fork23_available:
                print("Philosopher 2 has started eating") # print message for having both forks in hand
                eating(phil2)
                lock12.release() # release the forks
                lock23.release() # " " " 
                napping(phil2) #philosopher naps after the meal
            else:
                print("Philosopher 2 is starving...") # print message indicating both forks are not available
                time.sleep(0.2)
        elif num == phil3:
            p3_fork23_available = lock23.acquire(False) #non-blocking acquire()
            p3_fork34_available = lock34.acquire(False) #" " "
            if p3_fork23_available and p3_fork34_available:
                print("Philosopher 3 has started eating") # print message for having both forks in hand
                eating(phil3)
                lock23.release() # release the forks
                lock34.release() # " " " 
                napping(phil3) #philosopher naps after the meal
            else:
                print("Philosopher 3 is starving...") # print message indicating both forks are not available
                time.sleep(0.2)
        elif num == phil4:
            p4_fork34_available = lock34.acquire(False) #non-blocking acquire()
            p4_fork40_available = lock40.acquire(False) #" " "
            if p4_fork34_available and p4_fork40_available:
                print("Philosopher 4 has started eating") # print message for having both forks in hand
                eating(phil4)
                lock34.release() # release the forks
                lock40.release() # " " " 
                napping(phil4) #philosopher naps after the meal
            else:
                print("Philosopher 4 is starving...") # print message indicating both forks are not available
                time.sleep(0.2)        
    print('Program terminated. Press Shift + Enter in the cell to start philosophers` feast again.') 


threads = []
fork01 = threading.Lock() #the fork between philosophers 0 and 1
fork12 = threading.Lock() #the fork between philosophers 1 and 2
fork23 = threading.Lock() #the fork between philosophers 2 and 3
fork34 = threading.Lock() #the fork between philosophers 3 and 4
fork40 = threading.Lock() #the fork between philosophers 4 and 0

for i in range(5):
    t = threading.Thread(target=worker_phils, args=(fork01, fork12, fork23, fork34, fork40, i))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

Philosopher 0 has started eating
<locked _thread.lock object at 0xaf0431e8>
Philosopher 1 is starving...
Philosopher 2 is starving...
Philosopher 3 is starving...
Philosopher 4 is starving...
Philosopher 1 is starving...
Philosopher 2 is starving...
Philosopher 3 is starving...
Philosopher 4 is starving...
Philosopher 1 is starving...
Philosopher 2 is starving...
Philosopher 3 is starving...
Philosopher 4 is starving...
Philosopher 1 is starving...
Philosopher 2 is starving...
Philosopher 3 is starving...
Philosopher 4 is starving...
Philosopher 1 is starving...
Philosopher 2 is starving...
Philosopher 3 is starving...
Philosopher 4 is starving...
Philosopher 1 is starving...
Philosopher 2 is starving...
Philosopher 3 is starving...
<unlocked _thread.lock object at 0xaf0431e8>
Philosopher 4 is starving...
Philosopher 1 is starving...
Philosopher 2 is starving...
Philosopher 3 is starving...
Philosopher 4 is starving...
Philosopher 1 is starving...
Philosopher 2 is starving...
Philosoph